In [11]:
filename = "/home/begood/Videos/billiejean.mp4"

frames_dir = "./frames"

!rm -rf $frames_dir results
!mkdir $frames_dir
!ffmpeg -i $filename -vf fps=24 $frames_dir/out%d.png

In [12]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
import os
import argparse
import time
import cv2
import torch
import torch.nn as nn
from models import FastDVDnet
from fastdvdnet import denoise_seq_fastdvdnet
from utils import batch_psnr, init_logger_test, \
            variable_to_cv2_image, remove_dataparallel_wrapper, open_sequence, close_logger, get_imagenames
from matplotlib import pyplot as plt
import numpy as np
from tqdm import tqdm, trange

In [14]:
args={}
args['save_path'] = "results"
args['noise_sigma'] = 5 / 255
args['max_num_fr_per_seq'] = 5
#args['model_spatial_file'] = "model_spatial.pth"
model = "./model.pth"
args['suffix'] = ""
save_noisy = False
save_results = True
args['gray'] = False

In [15]:
args['cuda'] = torch.cuda.is_available()

In [16]:
NUM_IN_FR_EXT = 5 # temporal size of patch
#MC_ALGO = 'DeepFlow' # motion estimation algorithm
OUTIMGEXT = '.png' # output images format

# If save_path does not exist, create it
if not os.path.exists(args['save_path']):
    os.makedirs(args['save_path'])

In [17]:
# Sets data type according to CPU or GPU modes
if args['cuda']:
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

In [18]:
# Create models
model_temp = FastDVDnet(num_input_frames=NUM_IN_FR_EXT)
state_temp_dict = torch.load(model)

In [19]:
state_temp_dict = torch.load(model)
if args['cuda']:
    device_ids = [0]
    model_temp = nn.DataParallel(model_temp, device_ids=device_ids).cuda()
else:
    # CPU mode: remove the DataParallel wrapper
    state_temp_dict = remove_dataparallel_wrapper(state_temp_dict)
model_temp.load_state_dict(state_temp_dict)

In [20]:
model_temp.eval()
print("Eval mode set")

Eval mode set


In [21]:
#nb_file = len(os.listdir(frames_dir))
# Get ordered list of filenames
#files = get_imagenames(frames_dir)

def save_out_seq(seqclean, save_dir, sigmaval, start_idx):
    """Saves the denoised and noisy sequences under save_dir
    """
    seq_len = seqclean.size()[0]
    for idx in range(seq_len):
        save_img(seqclean, save_dir, sigmaval, start_idx,idx)

def save_img(seq, save_dir, sigmaval, start_idx, idx):
    out_name = os.path.join(save_dir,\
                ('n{}_DVDnet_{:0>15}').format(sigmaval, start_idx+idx) + OUTIMGEXT)

    outimg = variable_to_cv2_image(seq[idx].unsqueeze(dim=0))
    cv2.imwrite(out_name, outimg)

i= 0
while video.isOpened():
video.set(1, 1500)
ret, frame = video.read()
#gray = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

#Display the resulting frame
plt.imshow(frame)
plt.show()
#if i > 50:
#    break
#i+=1

In [22]:
video = cv2.VideoCapture(filename)
nb_frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
print("Amount of frames to process: {}".format(nb_frames))

Amount of frames to process: 8866


In [24]:
fps = video.get(cv2.CAP_PROP_FPS)
width  = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
fourcc = cv2.VideoWriter_fourcc('P','I','M','1')
out_filename = "{}-NONOISE2-DENOISED-noise{}-batch{}.mkv".format(filename,int(args['noise_sigma']*255),args['max_num_fr_per_seq'])
print("Frames per second: {}".format(fps))

Frames per second: 29.97002997002997


In [26]:
out = cv2.VideoWriter(out_filename, fourcc, fps, (width,height))
batch = args['max_num_fr_per_seq']
for fidx in trange(0, nb_frames, batch):
    
    max_num_fr=batch
    if fidx+batch > nb_frames:
        max_num_fr = nb_frames-fidx
        
    #print("Processing sequences from {} to {}".format(fidx, fidx+max_num_fr))
    seq, _, _ = open_sequence(video,\
                    args['gray'],\
                    expand_if_needed=False,\
                    start_index=fidx, max_num_fr=max_num_fr)
    # process data
    with torch.no_grad():
        seq = torch.from_numpy(seq).to(device)

        # Add noise
        noise = torch.empty_like(seq).normal_(mean=0, std=args['noise_sigma']).to(device)
        seqn = seq #+ noise
        noisestd = torch.FloatTensor([args['noise_sigma']]).to(device)
        #print("Denoising...")

        denframes = denoise_seq_fastdvdnet(seq=seqn,\
                                        noise_std=noisestd,\
                                        temp_psz=NUM_IN_FR_EXT,\
                                        model_temporal=model_temp)
    
    for idx in range(len(denframes)):
        outimg = variable_to_cv2_image(denframes[idx].unsqueeze(dim=0))
        out.write(outimg)
        
out.release()

100%|█████████▉| 1773/1774 [09:40<00:00,  3.05it/s]


IndexError: index 2 is out of bounds for dimension 0 with size 1

In [ ]:
# Save outputs
#save_out_seq(denframes, args['save_path'], int(noise_sigma*255))

!ffmpeg -framerate 24 \
    -pattern_type glob \
    -i 'results/*.png' \
    -c:v libx264  \
    -r 30 \
    -pix_fmt yuv420p10le \
    -map_chapters 0 \
    -preset fast \
    -crf 21 \
    -c:a copy \
    out.mkv

In [ ]:
seq, _, _ = open_sequence(video,\
                args['gray'],\
                expand_if_needed=False,\
                start_index=fidx, max_num_fr=max_num_fr)
# process data
with torch.no_grad():
    seq = torch.from_numpy(seq).to(device)

    # Add noise
    noise = torch.empty_like(seq).normal_(mean=0, std=args['noise_sigma']).to(device)
    seqn = seq #+ noise
    noisestd = torch.FloatTensor([args['noise_sigma']]).to(device)
    #print("Denoising...")

    denframes = denoise_seq_fastdvdnet(seq=seqn,\
                                    noise_std=noisestd,\
                                    temp_psz=NUM_IN_FR_EXT,\
                                    model_temporal=model_temp)

for idx in range(len(denframes)):
    outimg = variable_to_cv2_image(denframes[idx].unsqueeze(dim=0))
    out.write(outimg)